Varios cambios en el momento flojo. ahora vamos a hacerlo mas random en los clips y en flac para que se guarde de mejor manera y no pierda calidad de audio.

In [ ]:
!apt-get install -y ffmpeg
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Importamos las librerías necesarias
from google.colab import drive
from pydub import AudioSegment
import os
import random
import uuid

In [ ]:
# Función para montar Google Drive
def montar_drive():
    drive.mount('/content/drive')
    print("Google Drive montado correctamente.")

In [ ]:
# Función para descargar el video de YouTube como wav
def descargar_audio_youtube(url, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # Crear carpeta si no existe
    print(f"Descargando audio desde {url}...")
    # Descarga el audio en formato wav en la carpeta especificada
    !yt-dlp -x --audio-format wav -o "{output_folder}/%(title)s.%(ext)s" {url}
    print("Descarga completa.")

In [ ]:
# Función para dividir el archivo de audio en clips con duración y nombre aleatorios
def dividir_audio_en_clips(audio_file, min_duration_sec, max_duration_sec, output_folder_clips):
    # Carga el archivo de audio en formato FLAC
    audio = AudioSegment.from_file(audio_file, format="flac")

    # Crea la carpeta de clips si no existe
    os.makedirs(output_folder_clips, exist_ok=True)

    # Inicializamos la posición inicial
    start_time = 0

    # Divide el archivo en clips de duración aleatoria entre min_duration_sec y max_duration_sec
    while start_time < len(audio):
        # Genera una duración aleatoria para el clip dentro del rango especificado
        clip_duration_ms = random.randint(min_duration_sec * 1000, max_duration_sec * 1000)
        end_time = min(start_time + clip_duration_ms, len(audio))  # Evita pasar la duración del audio

        # Extrae el clip y le asigna un nombre único aleatorio
        clip = audio[start_time:end_time]
        clip_name = f"clip_{uuid.uuid4()}.wav"  # Nombre aleatorio único
        clip.export(os.path.join(output_folder_clips, clip_name), format="wav")
        print(f"Guardado: {clip_name} (Duración: {clip_duration_ms / 1000} segundos)")

        # Avanzamos al siguiente segmento
        start_time = end_time

    print(f"Proceso completo. Los clips se guardaron en {output_folder_clips}")


In [ ]:
# Función principal que combina todos los pasos
def procesar_video_youtube(url, min_duration_sec, max_duration_sec, folder_descargas, folder_clips):
    # Montamos Google Drive
    montar_drive()

    # Descargamos el video desde YouTube y lo convertimos a FLAC
    descargar_audio_youtube(url, folder_descargas)

    # Obtener el archivo de audio descargado (último archivo en la carpeta)
    audio_files = [f for f in os.listdir(folder_descargas) if f.endswith('.wav')]

    if not audio_files:
        print("No se encontró ningún archivo de audio en la carpeta de descargas.")
        return

    audio_file = max([os.path.join(folder_descargas, f) for f in audio_files], key=os.path.getctime)

    # Dividimos el audio en clips con duración aleatoria y nombre único
    dividir_audio_en_clips(audio_file, min_duration_sec, max_duration_sec, folder_clips)


In [ ]:
# Configuración de los parámetros y ejecución del flujo completo
url_video = "https://youtu.be/QKsRaelojKE?si=9jEwV11eUP5fPXQi"  # URL del video de YouTube

# Rutas en Google Drive
carpeta_descargas = "/content/drive/MyDrive/Proyecto SIC/videos_Audio_Pruebas"
carpeta_clips_IA = "/content/drive/MyDrive/Proyecto SIC/Voces IA"
carpeta_clips_Real = "/content/drive/MyDrive/Proyecto SIC/Voces reales"

# Definimos el rango de duración aleatoria para los clips
duracion_clip_min = 10  # Duración mínima en segundos
duracion_clip_max = 50  # Duración máxima en segundos

In [ ]:
# Ejecutamos el flujo completo en las carpetas de voces IA
procesar_video_youtube(url_video, duracion_clip_min, duracion_clip_max, carpeta_descargas, carpeta_clips_IA)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente.
Descargando audio desde https://youtu.be/QKsRaelojKE?si=9jEwV11eUP5fPXQi...
/bin/bash: line 1: yt-dlp: command not found
Descarga completa.
No se encontró ningún archivo de audio en la carpeta de descargas.


In [ ]:
# Ejecutamos el flujo completo en las carpetas de voces reales
procesar_video_youtube(url_video, duracion_clip_min, duracion_clip_max, carpeta_descargas, carpeta_clips_Real)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente.
Descargando audio desde https://youtu.be/gr2YhEbzWmc?si=VmDS44V9tO8hxBi1...
[youtube] Extracting URL: https://youtu.be/gr2YhEbzWmc?si=VmDS44V9tO8hxBi1
[youtube] gr2YhEbzWmc: Downloading webpage
[youtube] gr2YhEbzWmc: Downloading ios player API JSON
[youtube] gr2YhEbzWmc: Downloading mweb player API JSON
[youtube] gr2YhEbzWmc: Downloading m3u8 information
[info] gr2YhEbzWmc: Downloading 1 format(s): 251
[download] Destination: /content/drive/MyDrive/Proyecto SIC/videos_Audio_Pruebas/Modulo 6 ELIMINADO EMOCIONES NEGATIVAS.webm
[download] 100% of   22.87MiB in 00:00:00 at 38.68MiB/s
[ExtractAudio] Destination: /content/drive/MyDrive/Proyecto SIC/videos_Audio_Pruebas/Modulo 6 ELIMINADO EMOCIONES NEGATIVAS.flac
Deleting original file /content/drive/MyDrive/Proyecto SIC/videos_Audio_Pruebas/Modulo 6 ELIMINADO EMOCIONES NEGATIVAS.

In [ ]:
#funcion que cambie todos los archivos a .wav de las carpetas de drive
def convertir_a_wav(carpeta_drive):
    montar_drive()
    # Itera sobre todos los archivos en la carpeta especificada
    for archivo in os.listdir(carpeta_drive):
        # Obtiene la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta_drive, archivo)

        # Verifica si el archivo es .mp3 o .flac
        if archivo.endswith('.mp3') or archivo.endswith('.flac'):
            try:
                # Carga el archivo de audio según su formato, aumentando probesize y analyzeduration
                if archivo.endswith('.mp3'):
                    audio = AudioSegment.from_mp3(ruta_archivo)
                elif archivo.endswith('.flac'):
                    audio = AudioSegment.from_file(ruta_archivo, 'flac', parameters=["-probesize", "5000000", "-analyzeduration", "5000000"])

                # Define la nueva ruta para el archivo convertido a .wav
                nueva_ruta = os.path.join(carpeta_drive, os.path.splitext(archivo)[0] + '.wav')

                # Exporta el archivo a formato WAV
                audio.export(nueva_ruta, format='wav')
                print(f"Archivo convertido: {archivo} -> {os.path.basename(nueva_ruta)}")

                # Elimina el archivo original si es necesario
                os.remove(ruta_archivo)
                print(f"Archivo original eliminado: {archivo}")

            except Exception as e:
                print(f"Error al convertir {archivo}: {e}")

In [ ]:
#vamos a cambiar todos archivos de la carpeta carpeta_clips_IA
convertir_a_wav(carpeta_clips_IA)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente.
Archivo convertido: Ibai's Voice (7).mp3 -> Ibai's Voice (7).wav
Archivo original eliminado: Ibai's Voice (7).mp3
Archivo convertido: Ibai's Voice (8).mp3 -> Ibai's Voice (8).wav
Archivo original eliminado: Ibai's Voice (8).mp3
Archivo convertido: Ibai's Voice (2).mp3 -> Ibai's Voice (2).wav
Archivo original eliminado: Ibai's Voice (2).mp3
Archivo convertido: Ibai's Voice (3).mp3 -> Ibai's Voice (3).wav
Archivo original eliminado: Ibai's Voice (3).mp3
Archivo convertido: Felipe VI's Voice.mp3 -> Felipe VI's Voice.wav
Archivo original eliminado: Felipe VI's Voice.mp3
Archivo convertido: Felipe VI's Voice (3).mp3 -> Felipe VI's Voice (3).wav
Archivo original eliminado: Felipe VI's Voice (3).mp3
Archivo convertido: Felipe VI's Voice (2).mp3 -> Felipe VI's Voice (2).wav
Archivo original eliminado: Felipe VI's Voice (2).mp3
Arch

In [ ]:
#vamos a cambiar todos los archivos de la carpeta carpeta_clips_reales
convertir_a_wav(carpeta_clips_Real)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente.


In [ ]:
def eliminar_mp3_y_flac(carpeta_drive):
    # Itera sobre todos los archivos en la carpeta especificada
    for archivo in os.listdir(carpeta_drive):
        # Obtiene la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta_drive, archivo)

        # Verifica si el archivo es .mp3 o .flac
        if archivo.endswith('.mp3') or archivo.endswith('.flac'):
            try:
                # Elimina el archivo
                os.remove(ruta_archivo)
                print(f"Archivo eliminado: {archivo}")
            except Exception as e:
                print(f"No se pudo eliminar {archivo}: {e}")

In [ ]:
eliminar_mp3_y_flac(carpeta_clips_IA)
eliminar_mp3_y_flac(carpeta_clips_Real)